# Variational Classifier

In this notebook, we present an introduction to the work flow of quantum classifiers using **Pennylane**. This is supposed to be a simple guide, starting with the basics without requiring knowledge of classical machine learning.

## The Objective

Classifiers are used to create predictive models of classification problems. Our goal is to create **Variational Classifiers** that answers classification questions. 

Here, we build two models to answer two problems:
- Check the parity
- Breast Cancer Wisconsin https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original

# Parity check

For this problem, we are given $x$ and we have to determine $f(x)$, where $x$ is a bitstring $x \in \{0,1\}^{\otimes n}$ and $f(x)$ is the parity of $x$.
$$f(x) =
\begin{cases} 
0 & \text{if there is an even number of 1's in } x \\
1 & \text{else}
\end{cases}$$

Our goal is to create a quantum model and teach it how to solve $f(x)$ given any $x$.

You might be wondering what the quantum model looks like. Well, the model is a quantum circuit with adjustable parameters, *weights*. We train the model by testing different combinations of these weights or you can also call them variational parameters. The quantum circuit is called *ansatz*. The german word means an *educated guess*. Different problems might benefit from using a specific type of ansatz.

Choosing or developing the right ansatz isn't always simple as you need a quantum circuit with weights sensitive to your problem input, which may not always be straighforward.

For this notebook, we need the following imports

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

and we will run our quantum circuits in the following Pennylane simulator.

In [2]:
dev = qml.device("default.qubit")

### Ansatz

Our ansatz, as what you might expect, depends on the input length, `len(x)`. We will restrict ourselves to bitstrings of `len(x) = 4`.

*Note: Pennylane refers to qubits as wires*

In [3]:
# Ansatz
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)

I hope you noticed that we will use three weights in each layer, which are rotational angles. These three paramters are what we want to optimize in our ansatz. `layer` is the building block of our variational circuit. We can stack multiple layers as we optimize the weights or use a single layer. Our goal is to find optimal set of angles for a specific number of layers to evaluate the parity function.

### How to use the input $x$?

We use the following simple mapping from bitstring of length 4 to 4 qubits
$$x = 0101 \rightarrow |\psi\rangle = |0101\rangle.$$

In [4]:
def state_preparation(x: list):
    qml.BasisState(x, wires=[0, 1, 2, 3])

`BasisState` takes a list of integers $-$ zeros and ones $-$ to initialize qubits in the desired state. 

Right now, we have the functions we need to create our model $-$ the quantum circuit we want to optimize its parameters.

In [5]:
@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

Our circuit starts by preparing our qubits in the desired state. Then, add variational layers of our ansatz. In the end, we find the expectation of our 0th qubit in the Z basis, which has the range $[-1,1]$. Expectation values tells us the average of measurement results. The decorator `@qml.qnode(dev)` is used to tell `circuit` that it is preparing a quantum circuit (QNode to be precise in Pennylane's language) to run on the device we specified earlier.

We call our `circuit` using `variational_classifier` that prepares the quantum circuit with some paramaters `weights`. 

In [6]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

Due to the nature of our circuit, our model produces results that passes through the origin. So, we classically add an offset value `bias`, which we can think of as an additional weight we want to optimize, but classically.

How can we assess the results of our model? We use a `cost` function that compares our model's results to the actual results. This means that we are doing supervised learning $-$ we know the correct answers of the training dataset and we want the model to find the same results.

*Note: our goal is to minimize `cost`. `cost` tells us how far away our model is from getting the right answers.*

Our model produces `predictions`. There are multiple ways to find out how bad they are. One way is to sum up the absolute value of how far away each prediction is from the right answer, or even better, square these values. One benefit of squaring these values is to increase the penalty of producing really bad answers.

In [7]:
def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    # Pennylane doesn't allow us to do outside operations when evaluating the cost
    #       Cost functions need to be efficient and as direct as possible
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

In [8]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

Our `cost` function returns a single number, where the smaller is the better. Our goal is to create a model with minimum `cost` value.

`accuracy` just tells us the ratio of how many right answers our model got with a set of weights.

In [9]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

### Input Data

`X` includes the input, $x$, while `Y` includes the labels, $f(x)$.

$f(x)$ are given as $\{0, 1\}$, but we map them to $\{-1, 1\}$.

This mapping is important because we expect deterministic results of the expectation value. If the expectation value happened to be zero, then real life measurements would be random. How do we interpret the output? If the expectation value is not zero, this means there is a favorable outcome. We run the circuit multiple times to see which outcome is more likely, and that's our answer.

*Note: a quantum measurement gives us the eigenvalue of the qubit state.*

*$\{-1, 1\}$ are the eigenvalues of qubit measurement operators*

In [10]:
training_data = np.loadtxt("v1/train.txt", dtype=int)
X = np.array(training_data[:, :-1])
Y = np.array(training_data[:, -1])
Y = 2 * Y - 1 # shift label from {0, 1} to {-1, 1}
for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")


x = [0 0 0 1], y = 1
x = [0 0 1 0], y = 1
x = [0 1 0 0], y = 1
x = [0 1 0 1], y = -1
x = [0 1 1 0], y = -1
x = [0 1 1 1], y = 1
x = [1 0 0 0], y = 1
x = [1 0 0 1], y = -1
x = [1 0 1 1], y = 1
x = [1 1 1 1], y = -1


### Initial parameter preparation
In this step, we prepare a random starting point for our ansatz.

In [11]:
np.random.seed(0)
num_qubits = 4
num_layers = 2

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)

bias_init = np.array(0.5, requires_grad=True)

print(f"Initial Values of weights:\n{weights_init}")

print("Bias: ", bias_init)

Initial Values of weights:
[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]
Bias:  0.5


We use `requires_grad=True` because that is how we minimize the `cost` function. We find a gradient of `cost` with respect to our parameters.

We use `NesterovMomentumOptimizer`, an optimizer that takes into account momentum when finding the gradiant, which is a variant of the popular standard gradient descent optimizer. The `0.5` is the learning parameter speed. Steps of the generated new `weights` are proportional to the learning parameter. `batch_size` specifies how many training samples to use per iteration, from which we expand our gradient.

In [12]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 10

Now, we are ready to run our optimizer

In [13]:
weights = weights_init
bias = bias_init
for it in range(150):
    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 2.1172831 | Accuracy: 0.5000000
Iter:    2 | Cost: 2.3066813 | Accuracy: 0.5000000
Iter:    3 | Cost: 1.8111517 | Accuracy: 0.5000000
Iter:    4 | Cost: 1.9491181 | Accuracy: 0.5000000
Iter:    5 | Cost: 1.2568332 | Accuracy: 0.6000000
Iter:    6 | Cost: 1.1503186 | Accuracy: 0.8000000
Iter:    7 | Cost: 1.1016388 | Accuracy: 0.6000000
Iter:    8 | Cost: 0.5680689 | Accuracy: 0.8000000
Iter:    9 | Cost: 0.9816041 | Accuracy: 0.6000000
Iter:   10 | Cost: 1.2509711 | Accuracy: 0.6000000
Iter:   11 | Cost: 1.0835664 | Accuracy: 0.8000000
Iter:   12 | Cost: 1.0306310 | Accuracy: 0.6000000
Iter:   13 | Cost: 1.1885577 | Accuracy: 0.5000000
Iter:   14 | Cost: 1.1708716 | Accuracy: 0.5000000
Iter:   15 | Cost: 0.9885189 | Accuracy: 0.6000000
Iter:   16 | Cost: 1.0195433 | Accuracy: 0.6000000
Iter:   17 | Cost: 1.8276233 | Accuracy: 0.4000000
Iter:   18 | Cost: 0.9853444 | Accuracy: 0.6000000
Iter:   19 | Cost: 0.9926827 | Accuracy: 0.6000000
Iter:   20 | Cost: 1.1642655 | 

Note that we reached perfect accuracy on the training data.

### Test with new data
It is important to experiment on data our model hasn't seen befor. Sometimes, we might achieve high accuracy on the training data but create a useless model for new input. This is called overtraining, where we force our model to adapt to the training set not the actual problem.

In [14]:
data = np.loadtxt("v1/test.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

x = [0 0 0 0], y = -1, pred=-1.0
x = [0 0 1 1], y = -1, pred=-1.0
x = [1 0 1 0], y = -1, pred=-1.0
x = [1 1 1 0], y = 1, pred=1.0
x = [1 1 0 0], y = -1, pred=-1.0
x = [1 1 0 1], y = 1, pred=1.0
Accuracy on unseen data: 1.0


After creating this simple model, next, we create a more sophisticated model.

# Breast Cancer Wisconsin
Breast Cancer Wisconsin original dataset: https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original

To classify this dataset, we have to deal with a more complicated input. We will use a similar ansatz, but we have to find a way to initialize quantum states. So far, from what we know, our ansatz should be sensitive to the input to find patterns. Also, our state preparation routine should be expressive in the eyes of our ansatz. 

We will first define how to initialize our data. Then, we show our ansatz, after which, we produce our model and test it.

### Preprocessing
*Preparing initial quantum state*

For each entry, we have 9 inputs, which are called *features* in machine learning context. Refer to the source to inspect the data. They are all integers $ \in \{1, 2, ..., 10\}$
https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original


To encode 9 features in a quantum circuit, I follow a routine called *amplitude encoding*. There are other routines like *angle encoding*, which might be beneficial depending on the situation, but I will stick with amplitude encoding for simplicity.

Amplitude encoding routine maps each feature to a state in our system's Hilbert space. Since we have 9 features, this means we need at least 4 qubits to create a quantum system with a Hilbert space of dimension-16. So, we end up with 7 more states. Can we use the additional states to encode more information? Of course, we can extrapolate new features from the data we have. Note that we have to normalize our features befor encoding them. Normalization can erase information about the actual length of the input. To circumvent this issue, we pad each set of features with their magnitude. This way, the added feature stores some information about the actual magnitude of the input, which may be important in some cases.

Steps:
- Find the magnitudes of each features set
- Map each feature from integers $ \in \{1, 2, ..., 10\}$ to the range $[-1,1]$. We use negative values because amplitudes can be negative or even complex, but we don't make use of complex encoding in this tutorial. 
- Pad the magnitude and 6 zeros to create vectors with $dimension = 16 $
- Normalize 

As to our labels - the solutions - they are given as $\{2,4\}$, but again, we map them to $\{-1,1\}$ because qubit measurements yield unitary eignvalues, which are $\{-1,1\}$.

*Note: we replaced missing values with 5.5*

In [15]:
# There are some missing values we replaced them with 5.5
table =np.genfromtxt("v2/breast+cancer+wisconsin+original/breast-cancer-wisconsin.data", 
                     delimiter=',', missing_values='?', filling_values=5.5, dtype=float)

labelsL = table[:,-1] - 3 # Shift {2,4} -> {-1,1}
featuresL = table[:,1:-1]
# These are the magnitudes of each vector in featuresL
magL = np.linalg.norm(featuresL, axis=1)
# Map features [1, 10] -> [-1,1]
min_vals = np.min(featuresL, axis=0)
max_vals = np.max(featuresL, axis=0)
featuresL = 2 * (featuresL - min_vals) / (max_vals - min_vals) - 1

# We pad features in featuresL with the corresponding value from magL then add zeros
padding_0s = np.zeros((len(featuresL), 6)) 
padding_mag = np.array([np.full(1, mag) for mag in magL])
padding = np.c_[padding_mag, padding_0s]
padd_featuresL = np.c_[featuresL, padding]

# After padding, we normalize
norm = np.sqrt(np.sum(padd_featuresL**2, -1))
norm_padd_featuresL = (padd_featuresL.T / norm).T

# Split to training and test sets
np.random.seed(42)
indices = np.random.permutation(len(featuresL))
split_index = int(len(norm_padd_featuresL) * 0.8) # 80% for training
train_indices, test_indices = indices[:split_index], indices[split_index:]
featuresL_train = norm_padd_featuresL[train_indices]
featuresL_test = norm_padd_featuresL[test_indices]
labelsL_train, labelsL_test = labelsL[train_indices], labelsL[test_indices]


print("First entry features:", featuresL_train[0])
print("First entry label:", labelsL_train[0])

First entry features: [-0.13343587 -0.13343587 -0.10378346 -0.13343587 -0.07413104  0.
 -0.13343587 -0.13343587 -0.13343587  0.93643088  0.          0.
  0.          0.          0.          0.        ]
First entry label: -1.0


### Preparing Our Circuit
`AmplitudeEmbedding`

We showed that we are going to use 4 qubits to encode a state of $dimension = 16$ . Preparing such states on quantum devices isn't a simple taks, but Pennylane provides a shortcut we can use to prepare an arbitrary state `AmplitudeEmbedding`. When using real quantum devices, we need to generate our arbitrary state, which Pennylane still can do for us. As for our variational layer, we use the same layer we prepared earlier to manipulate 4 qubits.

In [16]:
num_qubits = 4
dev = qml.device("default.qubit", wires=num_qubits)

# Define the state preparation function
def state_preparation(features):
    num_qubits = 4
    qml.AmplitudeEmbedding(features, wires=[q for q in range(num_qubits)])

# Define the variational layer
def layer(layer_weights):
    for i in range(num_qubits):
        qml.Rot(*layer_weights[i], wires=i)
    for i in range(num_qubits):
        qml.CNOT(wires=[i, (i + 1) % num_qubits])

# Define the variational circuit
@qml.qnode(dev)
def circuit(weights, features):
    state_preparation(features)
    for layer_weights in weights:
        layer(layer_weights)
    return qml.expval(qml.PauliZ(0))

Using the same functions from before

In [17]:
def square_loss(labelsL, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labelsL - qml.math.stack(predictions)) ** 2)

def variational_classifier(weights, bias, features):
    prediction = np.array(circuit(weights, features))
    return prediction + bias

def cost(weights, bias, featuresL, labelsL):
    predictions = np.array([variational_classifier(weights, bias, features) for features in featuresL], requires_grad=True)
    return square_loss(labelsL, predictions)

def accuracy(labelsL, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labelsL, predictions))
    acc = acc / len(labelsL)
    return acc

### Run
Now, that we have everything ready, we can train our classifier. Of course, there are multiple parameters to play with here.

In [18]:
np.random.seed(99)
num_qubits = 4
num_layers = 6
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = NesterovMomentumOptimizer(0.05)
batch_size = 12

weights = weights_init
bias = bias_init
for it in range(40):
    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(featuresL_train), (batch_size,))
    X_batch = featuresL_train[batch_index]
    Y_batch = labelsL_train[batch_index]
    weights, bias = opt.step(cost, weights, bias, featuresL=X_batch, labelsL=Y_batch)
    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in featuresL_train]
    current_cost = cost(weights, bias, featuresL_train, labelsL_train)
    acc = accuracy(labelsL_train, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 1.8067526 | Accuracy: 0.3416816
Iter:    2 | Cost: 1.3001032 | Accuracy: 0.3416816
Iter:    3 | Cost: 0.6507062 | Accuracy: 0.8425760
Iter:    4 | Cost: 0.4554686 | Accuracy: 0.9338104
Iter:    5 | Cost: 0.4930879 | Accuracy: 0.6583184
Iter:    6 | Cost: 0.5434006 | Accuracy: 0.6583184
Iter:    7 | Cost: 0.5607458 | Accuracy: 0.6583184
Iter:    8 | Cost: 0.5131475 | Accuracy: 0.6583184
Iter:    9 | Cost: 0.4658143 | Accuracy: 0.6976744
Iter:   10 | Cost: 0.4412002 | Accuracy: 0.8747764
Iter:   11 | Cost: 0.4306980 | Accuracy: 0.9534884
Iter:   12 | Cost: 0.4410185 | Accuracy: 0.9731664
Iter:   13 | Cost: 0.4642192 | Accuracy: 0.9713775
Iter:   14 | Cost: 0.4993349 | Accuracy: 0.9409660
Iter:   15 | Cost: 0.4777656 | Accuracy: 0.9427549
Iter:   16 | Cost: 0.4420767 | Accuracy: 0.9624329
Iter:   17 | Cost: 0.4153720 | Accuracy: 0.9713775
Iter:   18 | Cost: 0.3838269 | Accuracy: 0.9731664
Iter:   19 | Cost: 0.3757428 | Accuracy: 0.9642218
Iter:   20 | Cost: 0.3876628 | 

### Analyze

We didn't reach 100% accuracy, but that is fine. Let's check the performance of our model on test data.

*If you want, you can compare our model to other baseline classical models in https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original*

In [19]:
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in featuresL_test]
acc = accuracy(labelsL_test, predictions)
print("Accuracy on test data:", f"{acc:0.4f}")

Accuracy on test data: 0.9571


That looks impressive, but is there anything we can do to improve our model? Let's take a closer look at `weights`.

In [20]:
weights

tensor([[[-1.32216623e-02, -1.00384775e-01, -8.87378024e-03],
         [ 2.01177902e-02,  5.41638977e-01,  5.81053787e-03],
         [-6.87783737e-03,  4.00790115e-01, -9.21608697e-03],
         [-2.58377066e-02, -1.96200056e-01,  6.46627550e-03]],

        [[ 1.37897366e-03,  5.53737071e-01,  5.89187059e-03],
         [-1.85980721e-02,  8.02055393e-02, -2.37206576e-03],
         [ 1.89052074e-03,  4.34496436e-01,  5.11194275e-03],
         [ 1.66626709e-02, -3.41487029e-01, -8.38294968e-04]],

        [[-1.00681470e-02, -6.64007460e-01, -9.84632737e-03],
         [-1.61546860e-02, -3.64667597e-01, -1.51402806e-03],
         [ 3.00764824e-03, -3.18093386e-01,  2.66194161e-02],
         [-4.17883909e-03, -9.16658713e-03, -5.03865929e-03]],

        [[-1.30847381e-02, -1.59851960e-01, -1.66705056e-02],
         [ 9.67586142e-03, -4.75333118e-01, -8.31843380e-04],
         [ 1.07620235e-02,  3.23709591e-03, -1.94755958e-02],
         [ 3.86263584e-03,  3.23025728e-01, -6.37953727e-03]],



As you can see, there are many small angles, which you might expect to be useless. Let's add a filter and round everything.

In [21]:
pr = .1 # filter small angles
weights_filter = weights.copy()
for idx0, w_l in enumerate(weights_filter):
    for idx1, l in enumerate(w_l):
        for idx2, ang in enumerate(l):
            if abs(ang) < pr:
                weights_filter[idx0][idx1][idx2] = 0
            elif abs(abs(ang) - np.pi/2) < pr:
                weights_filter[idx0][idx1][idx2] = np.pi/2 * abs(ang)/ang
            elif abs(abs(ang) - np.pi) < pr:
                weights_filter[idx0][idx1][idx2] = np.pi * abs(ang)/ang
            else: 
                weights_filter[idx0][idx1][idx2] = np.round(ang,1)
print("Weights after filtering and rounding:")
weights_filter

Weights after filtering and rounding:


tensor([[[ 0. , -0.1,  0. ],
         [ 0. ,  0.5,  0. ],
         [ 0. ,  0.4,  0. ],
         [ 0. , -0.2,  0. ]],

        [[ 0. ,  0.6,  0. ],
         [ 0. ,  0. ,  0. ],
         [ 0. ,  0.4,  0. ],
         [ 0. , -0.3,  0. ]],

        [[ 0. , -0.7,  0. ],
         [ 0. , -0.4,  0. ],
         [ 0. , -0.3,  0. ],
         [ 0. ,  0. ,  0. ]],

        [[ 0. , -0.2,  0. ],
         [ 0. , -0.5,  0. ],
         [ 0. ,  0. ,  0. ],
         [ 0. ,  0.3,  0. ]],

        [[ 0. , -0.2,  0. ],
         [ 0. ,  0. ,  0. ],
         [ 0. ,  0. ,  0. ],
         [ 0. ,  0.1,  0. ]],

        [[ 0. ,  0. ,  0. ],
         [ 0. ,  0.2,  0. ],
         [ 0. , -0.4,  0. ],
         [ 0. ,  0. ,  0. ]]], requires_grad=True)

In [22]:
predictions = [np.sign(variational_classifier(weights_filter, bias, x)) for x in featuresL_test]
acc_f = accuracy(labelsL_test, predictions)
print("Accuracy on test data with filter:", f"{acc_f:0.4f}", "\nAccuracy without a filter        :", f"{acc:0.4f}")

Accuracy on test data with filter: 0.9571 
Accuracy without a filter        : 0.9571


Well, this surprised me. Our model only does rotations around one axis - y - out of three options. I am not sure what this means, but this observation might allow us to train some models faster if we can ignore some variational parameters. Though, keep in mind that what appears to be useless parameters might play an important rule in helping our optimizer escape local minima.